In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [8]:
df = pd.read_csv(r"D:\Semester 7\Machine Learning\Assignment\dfFIFAdata_final_wtemp_pop_gdp.csv")
df.head()

,Unnamed: 0,Year,team,away_goals_conceded,away_score,away_games,home_score,home_goals_conceded,home_games,hs_mean,...,A_AttackS,H_DefenseS,A_DefenseS,WinCount,country,host_temp,team_temp,temp_diff,team_pop,team_gdp
0,0,1930,Argentina,4,2,1,16,5,4,2.277778,...,1.241379,0.775862,1.756098,4,Uruguay,17.384833,14.843417,-2.541417,20481779.0,NaN
1,1,1930,Brazil,0,4,1,1,2,1,2.277778,...,2.482759,1.241379,0.000000,1,Uruguay,17.384833,24.843833,7.459000,72179226.0,1.516557e+10
2,2,1930,Chile,3,1,1,4,0,2,2.277778,...,0.620690,0.000000,1.317073,2,Uruguay,17.384833,9.646333,-7.738500,8132990.0,4.110000e+09
3,3,1930,France,2,0,2,4,1,1,2.277778,...,0.000000,0.620690,0.439024,1,Uruguay,17.384833,13.168750,-4.216083,46621669.0,6.265147e+10
4,4,1930,Mexico,13,4,3,0,0,0,2.277778,...,0.827586,0.000000,1.902439,0,Uruguay,17.384833,20.613583,3.228750,37771859.0,1.304000e+10


In [9]:
# split the dataset in to train and test
traindf, testdf = train_test_split(df, test_size=0.33, random_state=42)

In [10]:
poisson_model= smf.glm(formula="WinCount ~ H_AttackS+A_AttackS+H_DefenseS+A_DefenseS+temp_diff+team_pop", data=traindf, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               WinCount   No. Observations:                 5203
Model:                            GLM   Df Residuals:                     5196
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5541.5
Date:                Tue, 17 Aug 2021   Deviance:                       1549.4
Time:                        16:09:57   Pearson chi2:                 1.30e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1271      0.047      2.701      0.007       0.035       0.219
H_AttackS      0.5519      0.018     29.834      0.000       0.516       0.588
A_AttackS      0.6077      0.019     32.777      0.000       0.571       0.644
H_DefenseS    -0.3963      0.017    -23.298      0.000      -0.430      -0.363
A_DefenseS    -0.9659      0.025    -38.409      0.000      -1.015      -0.917
temp_diff     -0.0057      0.001     -4.849      0.000      -0.008      -0.003
team_pop    5.035e-10   1.35e-10      3.736      0.000    2.39e-10    7.68e-10
==============================================================================
"""

In [11]:
y_pred = round(poisson_model.predict(testdf[['H_AttackS','A_AttackS','H_DefenseS','A_DefenseS','temp_diff','team_pop','team_gdp']]))

In [12]:
y_test = testdf['WinCount']
df_ = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_

,Actual,Predicted
6935,4,4.0
6437,1,1.0
6918,4,4.0
3685,2,2.0
3406,1,1.0
...,...,...
5848,0,0.0
5687,0,0.0
3535,1,1.0
1630,1,2.0


In [14]:
from sklearn.metrics import r2_score
y_true = df_.Actual
y_pred = df_.Predicted
r2_score(y_true, y_pred)

0.8703671073147512